In [73]:
import pandas as pd
import numpy as np 
from pandas_datareader import data
from func_pyeconomics import graf_plotly

In [74]:
ativos = ['CMIG4.SA','ELET3.SA','EGIE3.SA','EQTL3.SA','TAEE3.SA','TRPL4.SA', '^BVSP']

In [75]:
ativos

['CMIG4.SA',
 'ELET3.SA',
 'EGIE3.SA',
 'EQTL3.SA',
 'TAEE3.SA',
 'TRPL4.SA',
 '^BVSP']

### Normalização da carteira

In [76]:
def gera_carteira(ativos,ano_inicio, ano_final):
    lista_ativos = ativos
    dataframe = pd.DataFrame()
 
    for i in lista_ativos:
        frame = data.DataReader(i, data_source='yahoo', start=f'{ano_inicio}-01-01')
        frame = frame['Adj Close']
        frame = pd.DataFrame(frame)
        frame = frame.rename(columns={'Adj Close':f'{i}'})
        dataframe = pd.concat([frame, dataframe], axis = 1)
    return dataframe

In [77]:
def normaliza_carteira(dataframe):
    df_normalizado = pd.DataFrame()
    for i in dataframe.columns:
        df_normalizado[i] = dataframe[i] / dataframe[i][0]
    return df_normalizado

In [78]:
teste = gera_carteira(ativos,2000,2021)
teste

,^BVSP,TRPL4.SA,TAEE3.SA,EQTL3.SA,EGIE3.SA,ELET3.SA,CMIG4.SA
Date,,,,,,,
2000-01-03,16930.000000,2684.152100,NaN,NaN,NaN,8.838587,4352.254883
2000-01-04,15851.000000,2343.307617,NaN,NaN,NaN,8.411277,4134.304199
2000-01-05,16245.000000,2598.941406,NaN,NaN,NaN,8.748627,4156.772949
2000-01-06,16107.000000,2560.596191,NaN,NaN,NaN,8.546215,4211.823730
2000-01-07,16309.000000,2471.124268,NaN,NaN,NaN,8.681157,4224.181152
...,...,...,...,...,...,...,...
2021-04-20,120062.000000,26.000000,13.62,24.900000,41.799999,35.240002,13.500000
2021-04-22,119371.000000,25.920000,13.80,25.040001,41.669998,35.720001,13.450000
2021-04-23,120530.000000,26.000000,13.57,25.280001,42.209999,36.720001,13.500000


In [79]:
df_norm = normaliza_carteira(teste)
df_norm

,^BVSP,TRPL4.SA,TAEE3.SA,EQTL3.SA,EGIE3.SA,ELET3.SA,CMIG4.SA
Date,,,,,,,
2000-01-03,1.000000,1.000000,NaN,NaN,NaN,1.000000,1.000000
2000-01-04,0.936267,0.873016,NaN,NaN,NaN,0.951654,0.949922
2000-01-05,0.959539,0.968254,NaN,NaN,NaN,0.989822,0.955085
2000-01-06,0.951388,0.953968,NaN,NaN,NaN,0.966921,0.967734
2000-01-07,0.963320,0.920635,NaN,NaN,NaN,0.982188,0.970573
...,...,...,...,...,...,...,...
2021-04-20,7.091672,0.009686,NaN,NaN,NaN,3.987063,0.003102
2021-04-22,7.050856,0.009657,NaN,NaN,NaN,4.041370,0.003090
2021-04-23,7.119315,0.009686,NaN,NaN,NaN,4.154510,0.003102


In [80]:
benchmark = pd.DataFrame(df_norm['^BVSP'])
carteira = df_norm.drop(columns='^BVSP')
carteira['CARTEIRA'] = carteira.sum(axis=1) / len(carteira.columns)
carteira = pd.DataFrame(carteira['CARTEIRA'])
cart_bench = pd.concat([carteira, benchmark], axis = 1)

In [81]:
graf_plotly(cart_bench)

In [82]:
frame = data.DataReader('CMIG4.SA', data_source='yahoo', start='2000-01-01')
frame

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2000-01-03,10977.481445,10326.963867,10977.481445,10500.434570,1172481.0,4352.254883
2000-01-04,10706.431641,9974.599609,10706.431641,9974.599609,1311940.0,4134.304199
2000-01-05,10164.333984,9757.760742,10164.333984,10028.809570,1664644.0,4156.772949
2000-01-06,10297.148438,9839.075195,9895.996094,10161.624023,1970123.0,4211.823730
2000-01-07,10462.488281,10083.019531,10164.333984,10191.439453,1373183.0,4224.181152
...,...,...,...,...,...,...
2021-04-20,13.610000,12.970000,13.010000,13.500000,13665600.0,13.500000
2021-04-22,13.750000,13.350000,13.650000,13.450000,10860000.0,13.450000
2021-04-23,13.640000,13.300000,13.510000,13.500000,7326700.0,13.500000
